In [139]:
ssc.stop(stopSparkContext=False)

In [126]:
from pyspark.streaming import StreamingContext
# The "1" here is the number of seconds between microbatches:
ssc = StreamingContext(sc, 1)

# Required to be able to do state updates:
ssc.checkpoint("checkpoint")

In [127]:
# Assumes the stream is running on the same machine as the driver.
# That's not very common, so you'll probably change 'localhost'
# to something else.
sock = ssc.socketTextStream("orion03", 37600)

In [128]:
__base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
def encode(latitude, longitude, precision=12):
    """
    Encode a position given in float arguments latitude, longitude to
    a geohash which will have the character count precision.
    """
    lat_interval = (-90.0, 90.0)
    lon_interval = (-180.0, 180.0)
    geohash = []
    bits = [16, 8, 4, 2, 1]
    bit = 0
    ch = 0
    even = True
    while len(geohash) < precision:
        if even:
            mid = (lon_interval[0] + lon_interval[1]) / 2
            if longitude > mid:
                ch |= bits[bit]
                lon_interval = (mid, lon_interval[1])
            else:
                lon_interval = (lon_interval[0], mid)
        else:
            mid = (lat_interval[0] + lat_interval[1]) / 2
            if latitude > mid:
                ch |= bits[bit]
                lat_interval = (mid, lat_interval[1])
            else:
                lat_interval = (lat_interval[0], mid)
        even = not even
        if bit < 4:
            bit += 1
        else:
            geohash += __base32[ch]
            bit = 0
            ch = 0
    return ''.join(geohash)

In [129]:
def getLatLong(words):
    words.append(encode(float(words[1]),float(words[2]),3))
    return words

In [130]:
attr_name_idx = {0:"surface_temp", 1:"pressure", 2: "humidity", 3:"precipitation", 4:"visibility", 5:"wind_speed"}

In [131]:
from statistics import mean 
def calculateMean(attribute_val):
    return mean(attribute_val) 
    

In [132]:
# def addToMap(fields):
    
#     for i in range(6):
        
#         if not attr_name_idx[i] in mapGeohash[fields[6]]:
#             mapGeohash[fields[6]][attr_name_idx[i]] = []
        
#         mapGeohash[fields[6]][attr_name_idx[i]].append(fields[i])
        
        

In [133]:
def getReqdFields(words):
    '''
    geohash - 18, surface temperature 10, pressure 6, humidity 8, precipitation 13, visibility 15, wind speed 17
    '''
#     fields = [([float(words[10])],False),([float(words[6])],False),([float(words[8])],False),([float(words[13])],False),([float(words[15])],False),([float(words[17])],False)]
    fields = [[[float(words[10])],False],[[float(words[6])],False],[[float(words[8])],False],[[float(words[13])],False],[[float(words[15])],False],[[float(words[17])],False]]
    
#     if not words[18] in mapGeohash:
        
#         mapGeohash[words[18]] = {}
#         anomalies [words[18]] = {}
#         for i in range(6):
        
#             if not attr_name_idx[i] in anomalies[fields[6]]:
#                 anomalies[fields[6]][attr_name_idx[i]] = []

#             mapGeohash[fields[6]][attr_name_idx[i]].append(fields[i])

#             addToMap(fields)
        
#     elif len(mapGeohash[words[18]][attr_name_idx[0]])==50:
        
#         anomaly_detection(new_fields)
        
#     else:
#         addToMap(fields)
        

    return (words[18],fields)

In [134]:
# def anomaly_detection(new_fields):
#     geohash = new_fields[6]
#     for i in range(6):
#         mean = calculateMean(mapGeohash[fields[6]][attr_name_idx[i]])
#         is_anomaly = deviateChecker(mean, new_fields[i])
#         if not is_anomaly:
#             mapGeohash[fields[6]][attr_name_idx[i]].pop(0)
#             mapGeohash[fields[6]][attr_name_idx[i]].append(fields[i])
#         else:
#             anomalies[fields[6]][attr_name_idx[i]].append(fields[i])

def deviateChecker(mean, new):
    '''
    true means anomaly, i.e more than 120% of prev value
    '''
    return mean*1.2 < new

In [135]:
def computeVal(new, old):
    for i in range(6):
        if len(old[i][0]) == 10:
            mean = calculateMean(old[i][0])
            is_anomaly = deviateChecker(mean, new[i][0][0])
            if not is_anomaly:
                old[i][0].pop(0)
                old[i][0].append(new[i][0][0])
            else:
                old[i][1] = True
        else:
            old[i][0].append(new[i][0][0])

    return old

In [136]:
def print_anomaly(rdd):
    geohash = rdd[0]
    values = rdd[1]
    anom = " Anomaly "
    not_anom = " Not Anomaly "
    res = [geohash]
    for i in range(6):
        if values[i][1] == False:
            res.append(attr_name_idx[i] + " : " + not_anom )
        else:
            res.append(attr_name_idx[i] + " : " + anom)
    return res
    

In [137]:
words = sock.map(lambda line: line.split("	"))
words_with_geohash = words.map(lambda wordz: getLatLong(wordz))
weather_data = words_with_geohash.map(lambda x: getReqdFields(x))
reduced = weather_data.reduceByKey(lambda x, y: computeVal(y, x))
final_rdd = reduced.map(lambda x: print_anomaly(x))
# print(anomalies)



# reduced = weather_data.reduceByKey(lambda x, y: computeVal(y, x))
# distrib = reduced.updateStateByKey(computeVal)
# distrib.foreachRDD(plot_distribution)

# reduced = weather_data_mapped.reduceByKey(lambda x, y: y)
# words_with_geohash.foreachRDD(printRDD)

# Print out the distribution for good measure
# reduced.pprint()
final_rdd.pprint()
# words_with_geohash.pprint()
# filtered_data.pprint()
# wordsTemp = words.map(lambda x: getLatLong(x))

# Running this will start listening:
ssc.start()

-------------------------------------------
Time: 2020-12-10 00:09:13
-------------------------------------------

-------------------------------------------
Time: 2020-12-10 00:09:14
-------------------------------------------
['9hs | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Anomaly ', 'precipitation :  Anomaly ', 'visibility :  Not Anomaly ', 'wind_speed :  Anomaly ']
['f0q | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Anomaly ', 'precipitation :  Not Anomaly ', 'visibility :  Not Anomaly ', 'wind_speed :  Anomaly ']
['c10 | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Anomaly ', 'precipitation :  Anomaly ', 'visibility :  Not Anomaly ', 'wind_speed :  Anomaly ']
['cbh | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Anomaly ', 'precipitation :  Not Anomaly ', 'visibility :  Anomaly ', 'wind_speed :  Anomaly ']
['dqt | ', 'surface_temp :  Not Anomaly ', 'pre

-------------------------------------------
Time: 2020-12-10 00:09:22
-------------------------------------------
['bc9 | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Anomaly ', 'precipitation :  Not Anomaly ', 'visibility :  Not Anomaly ', 'wind_speed :  Not Anomaly ']
['9mq | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Anomaly ', 'precipitation :  Not Anomaly ', 'visibility :  Not Anomaly ', 'wind_speed :  Anomaly ']
['f6t | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Not Anomaly ', 'precipitation :  Not Anomaly ', 'visibility :  Not Anomaly ', 'wind_speed :  Not Anomaly ']
['c9b | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Not Anomaly ', 'precipitation :  Not Anomaly ', 'visibility :  Not Anomaly ', 'wind_speed :  Anomaly ']
['f0j | ', 'surface_temp :  Not Anomaly ', 'pressure :  Not Anomaly ', 'humidity :  Not Anomaly ', 'precipitation :  Anomaly ', 'visib